In [ ]:
!pip install transformers
!wget -O scibert_uncased.tar https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
!tar -xvf scibert_uncased.tar
import torch
from transformers import BertTokenizer, BertModel, AutoTokenizer, AutoModel, AutoModelForSequenceClassification
import pandas as pd
!pip install sentence_transformers
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LinearRegression
from scipy.stats import pearsonr
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from transformers import AutoTokenizer, AutoModel
import torch.nn.functional as F

from tensorflow.keras.layers import Input, Conv1D, GlobalMaxPooling1D, Dense, Dropout, Concatenate
from tensorflow.keras.models import Model

import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from transformers import AutoTokenizer, TFAutoModel
import tensorflow as tf


--2024-04-04 22:53:08--  https://s3-us-west-2.amazonaws.com/ai2-s2-research/scibert/huggingface_pytorch/scibert_scivocab_uncased.tar
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.92.204.240, 52.92.152.120, 52.218.182.104, ...
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.92.204.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 442460160 (422M) [application/x-tar]
Saving to: ‘scibert_uncased.tar’

scibert_uncased.tar 100%[===================>] 421.96M  19.6MB/s    in 23s     

2024-04-04 22:53:32 (18.2 MB/s) - ‘scibert_uncased.tar’ saved [442460160/442460160]

scibert_scivocab_uncased/
scibert_scivocab_uncased/vocab.txt
scibert_scivocab_uncased/pytorch_model.bin
scibert_scivocab_uncased/config.json
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 42.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823

In [ ]:
#reading features dataset
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/My Drive/Colab Notebooks/w266_project')
df = pd.read_csv(f"/content/drive/My Drive/Colab Notebooks/w266_project/join_df.csv", header = 0, sep="\t", index_col=0)


Mounted at /content/drive


In [ ]:
df

,Year Published,JIF,ISSN,Journal name,Eigenfactor,Authors,Author Full Name,Document Title,Publication Name,Author Keywords,...,ISO Source Abbreviation,Publication Date,Volume,Issue,Beginning Page,Ending Page,Digital Object Identifier (DOI),Page Count,Web of Science Categories,eISSN
2612,2000,2.526,0006-8993,NaN,NaN,"Soderstrom, K; Johnson, F","Soderstrom, K; Johnson, F",CB1 cannabinoid receptor expression in brain r...,BRAIN RESEARCH,cannabinoid; songbird; telencephalon; cognitiv...,...,Brain Res.,FEB 28,857.0,1-2,151,157,10.1016/S0006-8993(99)02393-8,7,Neurosciences,NaN
1063,2019,7.887,0161-5505,JOURNAL OF NUCLEAR MEDICINE,0.033,"Zhang, JJ; Kulkarni, HR; Singh, A; Schuchardt,...","Zhang, Jingjing; Kulkarni, Harshad R.; Singh, ...",<SUP>177</SUP>Lu-PSMA-617 Radioligand Therapy ...,JOURNAL OF NUCLEAR MEDICINE,prostate-specific membrane antigen; radioligan...,...,J. Nucl. Med.,NOV 1,60.0,11,1579,1586,10.2967/jnumed.118.223149,8,"Radiology, Nuclear Medicine & Medical Imaging",NaN
3627,2003,3.611,0007-1188,NaN,NaN,"Hornigold, DC; Mistry, R; Raymond, PD; Blank, ...","Hornigold, DC; Mistry, R; Raymond, PD; Blank, ...",Evidence for cross-talk between M<sub>2</sub> ...,BRITISH JOURNAL OF PHARMACOLOGY,muscarinic acetylcholine receptor; receptor-G ...,...,Br. J. Pharmacol.,APR,138.0,7.0,1340,1350,10.1038/sj.bjp.0705178,11,Pharmacology & Pharmacy,NaN
540,2005,2.477,0014-2999,NaN,NaN,"Kopanchuk, S; Veiksina, S; Petrovska, R; Mutul...","Kopanchuk, S; Veiksina, S; Petrovska, R; Mutul...",Co-operative regulation of ligand binding to m...,EUROPEAN JOURNAL OF PHARMACOLOGY,melanocortin receptor; radioligand binding; co...,...,Eur. J. Pharmacol.,APR 11,512.0,2-3,85,95,10.1016/j.ejphar.2005.02.021,11,Pharmacology & Pharmacy,NaN
2714,2020,10.057,0161-5505,JOURNAL OF NUCLEAR MEDICINE,0.033,"Coughlin, JM; Rubin, LH; Du, Y; Rowe, SP; Craw...","Coughlin, Jennifer M.; Rubin, Leah H.; Du, Yon...",High Availability of the α7-Nicotinic Acetylch...,JOURNAL OF NUCLEAR MEDICINE,nAChR; cholinergic; F-18-ASEM; dementia; PET,...,J. Nucl. Med.,MAR 1,61,3,423,426,10.2967/jnumed.119.230979,4,"Radiology, Nuclear Medicine & Medical Imaging",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2085,2005,3.940,1439-4227,CHEMBIOCHEM,NaN,"Niessen, KV; Höfner, G; Wanner, KT","Niessen, KV; Höfner, G; Wanner, KT",Competitive MS binding assays for dopamine D<s...,CHEMBIOCHEM,binding assays; dopamine receptors; drug scree...,...,ChemBioChem,OCT,6,10,1769,1775,10.1002/cbic.200500074,7,"Biochemistry & Molecular Biology; Chemistry, M...",NaN
98,2022,9.100,1619-7070,EUROPEAN JOURNAL OF NUCLEAR MEDICINE AND MOLEC...,0.027,"Bertoglio, D; Verhaeghe, J; Miranda, A; Wyffel...","Bertoglio, Daniele; Verhaeghe, Jeroen; Miranda...",Longitudinal preclinical evaluation of the nov...,EUROPEAN JOURNAL OF NUCLEAR MEDICINE AND MOLEC...,Neuroimaging; Biomarker; Animal model; mHTT; HD,...,Eur. J. Nucl. Med. Mol. Imaging,MAR,49,4,1166,1175,10.1007/s00259-021-05578-8,10,"Radiology, Nuclear Medicine & Medical Imaging",NaN
128,2014,0.481,1066-5285,NaN,NaN,"Sokolov, VB; Aksinenko, AY; Nikolaeva, NS; Gri...","Sokolov, V. B.; Aksinenko, A. Yu.; Nikolaeva, ...",Modification of biologically active amides and...,RUSSIAN CHEMICAL BULLETIN,tetrahydrocarbazoles; 5-fluoro-3-vinylpyridine...,...,Russ. Chem. Bull.,MAY,63,5,1137,1141,10.1007/s11172-014-0561-3,5,"Chemistry, Multidisciplinary",NaN
366,2000,2.869,0028-1298,NaN,NaN,"Corera, AT; Costentin, J; Bonnet, JJ","Corera, AT; Costentin, J; Bonnet, JJ",Binding of uptake blockers to the neuronal dop...,NAUNYN-SCHMIEDEBERGS ARCHIVES OF PHARMACOLOGY,dopamine transporter; ions; rat striatum; in v...,...,Naunyn-Schmiedebergs Arch. Pharmacol.,SEP,362,3,213,221,10.1007/s002100000280,9,Pharmacology & Pharmacy,NaN


In [ ]:
len(df)

4599

In [ ]:
!pip install -q transformers==4.37.2 --quiet
!pip install pydot --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 26.8 MB/s eta 0:00:00


In [ ]:
from sklearn.model_selection import train_test_split
from collections import Counter
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import BertTokenizer, TFBertModel, AutoTokenizer, AutoModel
import torch
from keras.callbacks import EarlyStopping
from sklearn.preprocessing import MinMaxScaler

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from transformers import BertTokenizer, TFBertModel, AutoTokenizer, AutoModel
import os
from sklearn.preprocessing import OneHotEncoder


In [ ]:
# Test/train split
X = np.vstack(df['Abstract'])
y = df['JIF']


In [ ]:
import numpy as np

# Determine the percentiles to divide the data into 20 equal parts
percentiles = np.linspace(0, 100, 20)

# Calculate the percentile boundaries
percentile_boundaries = np.percentile(y, percentiles)

# Assign each value in y to the nearest percentile boundary
y_percentile_rank = np.zeros_like(y)
for i in range(len(percentile_boundaries) - 1):
    lower_bound = percentile_boundaries[i]
    upper_bound = percentile_boundaries[i + 1]
    midpoint = (lower_bound + upper_bound) / 2
    y_percentile_rank[(y >= lower_bound) & (y < midpoint)] = lower_bound
    y_percentile_rank[(y >= midpoint) & (y <= upper_bound)] = upper_bound

# Normalize y_percentile_rank to range [0, 1]
y_percentile_rank /= np.max(percentile_boundaries)



In [ ]:
#applying one hot encoding to split data
from sklearn.preprocessing import OneHotEncoder

onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
y_onehot = onehot_encoder.fit_transform(y_percentile_rank.reshape(-1, 1))


X_train, X_val, y_onehot_train, y_onehot_val = train_test_split(X, y_onehot, test_size=0.2, random_state=42)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
# Load SciBERT tokenizer and model
model_checkpoint = "allenai/scibert_scivocab_uncased"
bert_tokenizer = BertTokenizer.from_pretrained(model_checkpoint)
bert_model = TFBertModel.from_pretrained(model_checkpoint, from_pt=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/228k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/442M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

In [ ]:
max_length = 500
train_encodings = bert_tokenizer(list(X_train[:,0]), truncation=True, padding=True, max_length=max_length, return_tensors='tf')
valid_encodings = bert_tokenizer(list(X_val[:,0]), truncation=True, padding=True, max_length=max_length, return_tensors='tf')


In [ ]:
#3 with reduced kernals
import tensorflow as tf
def create_bert_cnn(checkpoint = bert_model,
                                 hidden_size = 50,
                                 dropout=0.3,
                                 learning_rate=0.00005):
    """
    Build a simple classification model with BERT. Use the Pooler Output for classification purposes.
    """
    ### YOUR CODE HERE
    bert_model.trainable = False

    input_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='input_ids_layer')
    token_type_ids = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='token_type_ids_layer')
    attention_mask = tf.keras.layers.Input(shape=(max_length,), dtype=tf.int64, name='attention_mask_layer')

    bert_inputs = {'input_ids': input_ids,
                   'token_type_ids': token_type_ids,
                   'attention_mask': attention_mask}

    bert_out = bert_model(bert_inputs)

    #token = bert_out[1] #pooler_token
    token = bert_out[0][:, 0, :] #cls token

     # Add dense layers after BERT




    # Adding CNN layers
    cnn_input = tf.keras.layers.Reshape((768,1))(token)  # Reshape for token size
    conv1 = tf.keras.layers.Conv1D(32, 3, activation='relu')(cnn_input)
    pool1 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv1)
    conv2 = tf.keras.layers.Conv1D(64, 3, activation='relu')(pool1)
    pool2 = tf.keras.layers.MaxPooling1D(pool_size=2)(conv2)
    flatten = tf.keras.layers.Flatten()(pool2)

    hidden1 = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer1')(flatten)
    hidden1 = tf.keras.layers.Dropout(dropout)(hidden1)
    hidden2 = tf.keras.layers.Dense(hidden_size, activation='relu', name='hidden_layer2')(hidden1)
    hidden2 = tf.keras.layers.Dropout(dropout)(hidden2)

    # Output layer
    output = tf.keras.layers.Dense(20, activation='softmax', name='output_layer')(hidden2) # change to softmax


    # Define model
    model = tf.keras.Model(inputs=[input_ids, token_type_ids, attention_mask], outputs=output)

    # Compile model
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
              loss='categorical_crossentropy',
              metrics=['accuracy'])


    return model



In [ ]:
scibert_model_cnn = create_bert_cnn(checkpoint=bert_model, hidden_size = 768, dropout=0.2)

In [ ]:
scibert_model_cnn.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 attention_mask_layer (Inpu  [(None, 500)]                0         []                            
 tLayer)                                                                                          
                                                                                                  
 input_ids_layer (InputLaye  [(None, 500)]                0         []                            
 r)                                                                                               
                                                                                                  
 token_type_ids_layer (Inpu  [(None, 500)]                0         []                            
 tLayer)                                                                                      

In [ ]:
#training model
early_stopping = EarlyStopping(monitor='accuracy',
                               patience=5,
                               mode = "auto",
                               restore_best_weights=True)

bert_model_history = scibert_model_cnn.fit(
    x=[train_encodings.input_ids, train_encodings.token_type_ids, train_encodings.attention_mask],
    y=y_onehot_train,  # Use percentile rank as the target variable
    validation_data=(
        [valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask],
        y_onehot_val  # Use percentile rank for validation data
    ),
    batch_size=100,
    epochs=200, callbacks=[early_stopping]
)


Epoch 1/200
37/37 [==============================] - 91s 2s/step - loss: 2.9347 - accuracy: 0.0856 - val_loss: 2.8788 - val_accuracy: 0.1141
Epoch 2/200
37/37 [==============================] - 68s 2s/step - loss: 2.8513 - accuracy: 0.1261 - val_loss: 2.7967 - val_accuracy: 0.1511
Epoch 3/200
37/37 [==============================] - 68s 2s/step - loss: 2.7751 - accuracy: 0.1386 - val_loss: 2.7148 - val_accuracy: 0.1685
Epoch 4/200
37/37 [==============================] - 68s 2s/step - loss: 2.7127 - accuracy: 0.1460 - val_loss: 2.6690 - val_accuracy: 0.1674
Epoch 5/200
37/37 [==============================] - 68s 2s/step - loss: 2.6746 - accuracy: 0.1680 - val_loss: 2.6359 - val_accuracy: 0.1859
Epoch 6/200
37/37 [==============================] - 68s 2s/step - loss: 2.6243 - accuracy: 0.1764 - val_loss: 2.6092 - val_accuracy: 0.1946
Epoch 7/200
37/37 [==============================] - 68s 2s/step - loss: 2.6006 - accuracy: 0.1900 - val_loss: 2.5821 - val_accuracy: 0.1891
Epoch 8/200
3

In [ ]:
train_encodings.input_ids.shape


train_encodings.token_type_ids.shape

train_encodings.attention_mask.shape

TensorShape([3679, 500])

In [ ]:
pred= scibert_model_cnn.predict([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask])

pred

In [ ]:
# running with 20 classes
from scipy.stats import pearsonr

# Make predictions on the validation data
y_pred = scibert_model_cnn.predict([valid_encodings.input_ids, valid_encodings.token_type_ids, valid_encodings.attention_mask])

# Compute the Pearson correlation coefficient
corr, p_value = pearsonr(y_pred.flatten(), y_onehot_val.flatten())

print("Pearson correlation coefficient:", corr)
print("P-value:", p_value)


29/29 [==============================] - 14s 381ms/step
Pearson correlation coefficient: 0.24624313768530318
P-value: 3.0083498022371757e-252
